In [15]:
import numpy as np
import pypianoroll
import os

dic = np.load('interpolation.npy', allow_pickle=True)
dic = dic.item()
print(dic['preda'].shape)
print(dic['predb'].shape)

(1, 160)
(1, 160)


In [4]:
"""
Arguments:
    data: ndarray (time_steps,)
Return:
    pianoroll: ndarray (time_steps, 128)
"""
def data2pianoroll(data):
    res = np.zeros((len(data), 128), dtype=bool)
    prev = -1
    for i,x in enumerate(data):
        if x < 128:
            res[i][x] = True
            prev = x
        elif x==128:
            res[i][prev] = True
    return res
    

"""
Arguments:
    measure: ndarray (160,)
Return:
    pianoroll: pypianoroll.Track (time_steps, 128)
"""
def merge(measure):
    parts = np.split(measure, int(len(measure)/16))
    parts = list(map(lambda x: data2pianoroll(x), parts))
    pianoroll = np.logical_or.reduce(parts)
    multitrack = pypianoroll.Multitrack(beat_resolution=4)
    multitrack.append_track(pypianoroll.Track(pianoroll))
    return multitrack

In [16]:
pypianoroll.write(merge(dic['preda'][0]), 'preda.mid')
pypianoroll.write(merge(dic['predb'][0]), 'predb.mid')
for j in range(7):
    pypianoroll.write(merge(dic['inter'][j][0]),str(j) + '.mid')

In [8]:
for j in range(100):
    os.mkdir("reconstruction_2_poly/" + str(j))
    for i in range(len(dic[j]["gd"])):
        pypianoroll.write(merge(dic[j]['gd'][i]), "reconstruction_2_poly/" + str(j) + "/" + str(i) +  '_pred.mid')
        pypianoroll.write(merge(dic[j]['pred'][i]), "reconstruction_2_poly/" + str(j) + "/"  + str(i) + '_truth.mid')